# **Import necessary libraries**



In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import os
import random as rand
import string
from difflib import get_close_matches
#!pip install openpyxl # because the docker images on our cocalc servers didn't have it installed for some weird, unknown reason.

# **Define necessary custom functions that will be useful when cleaning the dataset**

In [7]:
# Create a function to find outliers using IQR
def find_outliers_IQR(df):
    # this function takes a df or only a column and calculates outliers for each value based on IQR

    q1 = df.quantile(0.25)  # variable for the first quartile

    q3 = df.quantile(0.75)  # variable for the third quartile

    IQR = q3 - q1  # calculate the IQR

    outliers = df[
        ((df < (q1 - 1.5 * IQR)) | (df > (q3 + 1.5 * IQR)))
    ]  # calculate outliers for every value and put them in a series

    return outliers


# Range function
def range_column(df):
    max = df.max()
    min = df.min()
    range = max - min
    return range


# (NA/Null) Value Percentage Calculator
def naValues(df):

    for column in df:

        total_values = len(df.index)  # number of all values in a column

        total_garb = (
            df[column].isna().sum()
        )  # number off all NA/Null values (apparently null and na are same in pandas, df.isnull() == df.isna())

        garb_perc = (total_garb * 100) // total_values

        print(column, " has total of ", total_garb, " NA/Null values")
        print("NA/Null percentage of ", column, " is ", garb_perc, "% \n")


# A custom function to replace special characters with corresponding letters
def replaceSpecialChars(text):
    # Define character replacements
    char_replacements = {
        "@": "a",
        "3": "e",
        "1": "i",
        "0": "o",
        "!": "i",
        "#": "h",
        "$": "s",
        "5": "s",
        "7": "t",
        "9": "g",
    }
    # If the input is not a string, return it as-is
    if not isinstance(text, str):
        return text
    # remove empty spaces
    text = text.strip()
    # Replace each special character in the text
    for char, replacement in char_replacements.items():
        text = text.replace(char, replacement)

    return text


# A function that manipulates strings in columns.
def str_basicclean(
    df, column="", style="cap", sp_char=string.whitespace, sp_replace=False
):  # capitalizes strings and strips whitespaces by default, it can strip anything if provided as argument.
    try:
        if df[column].dtype == "object":

            if type(sp_char) == str:
                try:
                    df[column] = df[column].str.strip()
                    df[column] = df[column].str.strip(sp_char)
                except:
                    print("ERROR: Strip function did not work properly")
            else:
                df[column] = df[column].str.strip()

            if style == "up":
                df[column] = df[column].str.upper()
            elif style == "low":
                df[column] = df[column].str.lower()
            elif style == "cap":
                df[column] = df[column].str.capitalize()

            if sp_replace == True:
                df[column] = df[column].apply(replaceSpecialChars)
    except:
        print(f"ERROR: Data Type of the {column} is not object.")
        return 0
    return df[column]


# A function to list all the unique values
def printUniqueValues(df, columnKey, sort_function=None):
    """
    Print unique values of a column from a df, with optional sorting.

    Parameters:
    df (pd.df): The df containing the column.
    columnKey (str): The key of the column whose unique values are to be printed.
    sort_function (callable, optional): A function to sort the unique values. Defaults to None.
    """
    unique_values = df[columnKey].unique().tolist()

    # Apply sorting if a sort function is provided
    if sort_function:
        unique_values = sorted(unique_values, key=sort_function)

    print(unique_values)


# A custom function to fill the empty or nan values with a selected mod
def num_filler(df, column="", mode="median"):
    if df[column].dtype == "object":
        print(f"ERROR: The data-type of column {column} is not numerical!")
    else:
        match mode:
            case "median":
                df_median = df[column].median()
                df[column] = df[column].fillna(df_median)
                return df[column]
            case "mean":
                df_mean = df[column].mean()
                df[column] = df[column].fillna(df_mean)
                return df[column]
            case "mod":
                df_mod = df[column].mod()
                df[column] = df[column].fillna(df_mod)
                return df[column]
            case "prob":
                value_counts = df[column].value_counts()
                probabilities = value_counts / value_counts.sum()

                # Randomly generate values for NaN based on the probabilities
                random_values = np.random.choice(
                    value_counts.index, size=df[column].isna().sum(), p=probabilities
                )

                # Fill the empty values with the randomly generated values
                df.loc[df[column].isna(), column] = random_values
                return df[column]
            case _:
                print(
                    "ERROR: mod is unvalid. Please choose one of the mods below:\n median(default), mean, mod, prob"
                )
                return df[column]


import pandas as pd
from difflib import get_close_matches


def handle_incomplete_data(value, valid_values, default="Unknown", cutoff=0.4):
    if not isinstance(value, str):
        return default

    # Strip and attempt to find a close match
    value = value.strip()
    matches = get_close_matches(value, valid_values, n=1, cutoff=cutoff)
    if matches:
        return matches[0]
    else:
        return default


def process_column(df, column, valid_values, default="Unknown", cutoff=0.4):

    return df[column].apply(
        lambda x: handle_incomplete_data(x, valid_values, default, cutoff)
    )


def save_unique_values_to_file(df, column, filename):
    unique_values = df[column].unique().tolist()
    try:
        os.makedirs(os.path.dirname(filename), exist_ok=True)
    except OSError as e:
        print(f"Error creating directory: {e}")
        return
    with open(filename, "w") as file:
        for value in unique_values:
            file.write(f"{value}\n")


def removeSpecialChars(df, columnKey, chars=[]):
    """
    Remove specified special characters from a column in the DataFrame. This function will be used for cleaning columns with basic strings with unwanted special characters; Not for replacing special characters with corresponding letters.
    """
    for char in chars:
        df[columnKey] = df[columnKey].str.replace(char, "", regex=False)

    return df


def removePartialStrings(df, columnKey, substrings=[]):
    """
    Remove specified substrings from a column in the DataFrame. This function will be used for cleaning columns with basic strings with unwanted substrings.
    """
    for substring in substrings:
        df[columnKey] = df[columnKey].str.replace(substring, "", regex=False)

    return df


from difflib import SequenceMatcher


def findFuzzyGroups(df, column, threshold=0.85):
    values = df[column].dropna().astype(str).unique().tolist()
    visited = set()
    groups = []

    for i, v1 in enumerate(values):
        if v1 in visited:
            continue

        group = [v1]
        visited.add(v1)

        for v2 in values[i + 1 :]:
            if v2 in visited:
                continue

            similarity = SequenceMatcher(None, v1.lower(), v2.lower()).ratio()

            if similarity >= threshold:
                group.append(v2)
                visited.add(v2)

        if len(group) > 1:
            groups.append(group)

    return groups


def capitalize_words(df, column):
    df[column] = df[column].astype(str).str.lower().str.title()
    return df

# **Import the raw data via pandas**


In [8]:
#Import the raw dataset via pandas
df= pd.read_csv('turkey_university_exam_score_dirty_v2.csv', sep=",") #sep=";" is needed since the seperator of the csv file is semicolon instead of comma

FileNotFoundError: [Errno 2] No such file or directory: 'turkey_university_exam_score_dirty_v2.csv'

# **Display descriptive and technical info about the raw, dirty data**

In [ ]:
df.describe().transpose().round(3)

The MİN and MAX values are looking good

In [ ]:
#display the head and tail
display(df.head(), df.tail())

##### *Column names should be fixed. Also, values should be unified on object type columns.*

In [ ]:
#display info about the df
df.info()

***Almost all of the columns should be renamed, according to the data description:***

- *'student id'* to *'StudentID'*
- *'EXAM_year'* to *'ExamYear'*
- *' UniversityName'* to *'UniversityName'*  
- *'Home-City'* to *'HomeCity'*
- *'major'* to *'Major'*
- *'GENDER'* to *'Gender'*
- *'SocioEconomic_status'* to *'SocioEconomicStatus'*
- *'PartTime job'* to *'PartTimeJob'*
- *'StudyHours/Week'* to *'StudyHoursPerWeek'*  
- *'Tutoring_Hours_per_Week'* to *'TutoringHoursPerWeek'*
- *'attendance rate'* to *'AttendanceRate'*
- *'priorGPA'* to *'PriorGPA'*

***Also some data-types seem to be wrong:***
- *'Enrollmentyear' should be converted from ```float64``` to ```int```*
- *'ExamYear' should be converted from ```float64``` to ```int```*
- *'SleepHours' should be converted from ```object``` to ```float64```*


***Other columns look consistent with their dtypes and descriptions.***



##### *Check for duplicates and unique values (will check again later).*

In [ ]:
#examine duplicates
display(df.duplicated().sum(), df[df.duplicated(keep=False)])

***There are some duplicates. Lets investigate further.***

In [ ]:
#check for duplicate rows without considering 'student id'
dup_rows = df[df.duplicated(keep=False)]
dup_rows.drop(columns=['student id']).duplicated().sum()


There are 10 duplicate pairs of rows that are identical in every column except student id. We will keep one of each pair and drop the other

In [ ]:
# Keep one of each pair and drop the other.
df = df.drop_duplicates(subset=df.columns.difference(['student id']), keep='first')

In [ ]:
#Check for the number of unique values
print(df.nunique())
print("\n")

#save the unique values for each columns to specify the problems
output_dir = "unique_values"
os.makedirs(output_dir, exist_ok=True)  # create dir if not exists


#extract the unique values and save for each column
for column in df.columns:
    unique_values = df[column].unique()
    column_name = column.replace(" ", "_").replace("/", "_").lower()
    file_path = os.path.join(output_dir, f"{column_name}.txt") # only will be used for specify the uniqe values of columns, so we dont need to save as csv,json etc.
    #save unique values
    with open(file_path, "w") as file:
        for value in unique_values:
            file.write(f"{value}\n")
    
    print(f"Unique values for '{column}' saved to {file_path}")

In [ ]:
# calculate the amount and percentage of NA/Null values on each column using our custom function, naValues
naValues(df)

***Although the missing data percentages are small we will not remove them directly.***

***Data seems to be imported fine. Column headers are values. We don't have multiple variables in a column. Both rows and columns hold values. Every column has the same unit in itself.***

# **Cleaning & Tidying**

## **Make title names more appropriate**



In [5]:
#column names from dataset description. ordered by dataset.csv columns
column_names = [
    "Student_ID",
    "Enrollment_Year",
    "Exam_Year",
    "University_Name",
    "Home_City",
    "Major",
    "Gender",
    "Scholarship_Status",
    "Socioeconomic_Status",
    "PartTime_Job",
    "Study_Hours_per_Week",
    "Tutoring_Hours_per_Week",
    "Attendance_Rate",
    "Sleep_Hours_per_Night",
    "Prior_GPA",
    "Exam_Score"
]
#rename the column names by dataset description
df.columns = column_names
df.head()


# Convert columns to appropriate data types
df['Sleep_Hours_per_Night'] = pd.to_numeric(df['Sleep_Hours_per_Night'], errors='coerce')

df['Exam_Year'] = pd.to_numeric(df['Exam_Year'], errors='coerce').astype('Int64')



NameError: name 'df' is not defined

In [ ]:
df.describe().transpose().round(3)

### **Enrollment_Year Column**

In [ ]:
display(df['Enrollment_Year'].describe())

In [ ]:
"""Clean 'Enrollment_Year' column"""

def cleanYearColumns(df, columnKey="Enrollment_Year", startRange=2015, endRange=2021):
    """
    Clean the 'Enrollment_Year' and 'Exam_Year' columns in one function (Since their cleaning process are identical):
    - Ensure values are within the range 2015 to 2021.
    - Handle missing values by filling them with the overall median year.
    """

    df[columnKey] = pd.to_numeric(df[columnKey], errors='coerce').astype('Int64')
    # Clip values to the valid range (2000 to 2024)
    df[columnKey] = df[columnKey].clip(startRange, endRange)
    
    # Fill missing values with the overall median year
    num_filler(df,columnKey,'median')
    

    
    return df


cleanYearColumns(df,"Enrollment_Year",2015,2021)
cleanYearColumns(df,"Exam_Year",2015,2021)
"""
- print unique values of column
- since our years are not a big data, we can manually validate our cleaned column data
"""
printUniqueValues(df,"Enrollment_Year",sort_function=lambda x: x)
printUniqueValues(df,"Exam_Year",sort_function=lambda x: x)
#saveFile("processing_dataset.csv",df_years_cleaned.to_csv(index=False))


### **University_Name Column**

In [ ]:
display(df["University_Name"].describe())

***It can be clearly seen that there are same university names with letter capitalization problems or used short names of universities. There is also some special characters that is not used for any letter replacement***

In [ ]:
str_basicclean(df,'University_Name',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of
print(len(df['University_Name'].unique()))
#Now we have 27 unique university names after lowercasing and stripping whitespaces, lets save them to a file for visual inspection
save_unique_values_to_file(df, 'University_Name', 'processing/university_name.txt')

After visual inspection, we decided to remove special characters

In [ ]:
#Remove special characters '!' and '#' from 'University_Name' column
removeSpecialChars(df, 'University_Name', chars=['!','#'])
save_unique_values_to_file(df, 'University_Name', 'processing/university_name.txt')
print(len(df['University_Name'].unique()))

After previous cleaning steps, we are succesfully reduced  68 unique values to 17. Now we visually identify that there are some typos "university" or "university" so lets remove that suffixes.

In [ ]:
# remove 'university' or 'universty' suffixes from names
removePartialStrings(df, 'University_Name', substrings=[' university', ' universty'])

save_unique_values_to_file(df, 'University_Name', 'processing/university_name.txt')
print(len(df['University_Name'].unique()))

We can manually map the remained 13 unique values and fill the NaN rows with "Other Universities" 

In [ ]:
#manually created dictionary of correct university names based on visual inspection of unique values
UniversityNames = {
    "yildiz technical": "Yildiz Technical University",
    'bilkent': "Bilkent University",
    'hacettepe': "Hacettepe University",
    'istanbul': "Istanbul University",
    'koc': "Koc University",
    'bogazici': "Bogazici University",
    'ankara': "Ankara University",
    'ege': "Ege University",
    'gazi': "Gazi University",
    'metu': "Middle East Technical University",
    'dokuz eylul': "Dokuz Eylul University",
    'itu': "Istanbul Technical University",
}
#process the 'University_Name' column using the manually created dictionary
df['University_Name'] = process_column(df, 'University_Name', list(UniversityNames.keys()), default='Other Universities', cutoff=0.4).map(UniversityNames).fillna('Other Universities')
save_unique_values_to_file(df, 'University_Name', 'processing/university_name.txt')

In [ ]:
df['University_Name'].describe()

University_Name column cleared

### **Home_City Column**

In [ ]:
display(df['Home_City'].describe())
display(df['Home_City'].unique())
save_unique_values_to_file(df, 'Home_City', 'processing/home_city.txt')

***Home_City column also has the same problems with University_Name column. So the same steps will be follwed.***

In [ ]:
str_basicclean(df,'Home_City',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of
print(len(df['Home_City'].unique()))
#Now we have 27 unique university names after lowercasing and stripping whitespaces, lets save them to a file for visual inspection
save_unique_values_to_file(df, 'Home_City', 'processing/home_city.txt')

In [ ]:
#Remove special characters '!' and '#' from 'Home_City' column
removeSpecialChars(df, 'Home_City', chars=['!','#'])
save_unique_values_to_file(df, 'Home_City', 'processing/home_city.txt')
print(len(df['Home_City'].unique()))

In [ ]:

similar_cities = findFuzzyGroups(df, 'Home_City', threshold=0.7)
print(f"Similar cities to {similar_cities}")
        

We don't have much typos so lets map them and fill the NaN values with "Other Cities"

In [ ]:
# We don't have much typos so lets map them and fill the NaN values with "Other Cities" using similar_cities varaible:  [['ankara', 'ankraa'], ['izmir', 'i̇zmir'], ['istanbul', 'istambul'], ['diyarbakir', 'diyarbakır']]

df["Home_City"] = (
    df["Home_City"]
    .replace(
        {
            "ankraa": "ankara",
            "i̇zmir": "izmir",
            "istambul": "istanbul",
            "diyarbakır": "diyarbakir",
        }
    )
    .fillna("Other Cities")
)
df["Home_City"] = str_basicclean(df, "Home_City", style="cap")

save_unique_values_to_file(df, "Home_City", "processing/home_city.txt")
df["Home_City"].describe()
df['Home_City'].unique()

***After visually inspecting the Home_City column, we can conclude its cleaned***

### **Major Column**

In [ ]:
display(df['Major'].describe())
display(df['Major'].unique())
save_unique_values_to_file(df, 'Major', 'processing/major.txt')

***Major column also has the same problems with University_Name and Home_City column. So the similar steps will be followed.***

In [ ]:
str_basicclean(df,'Major',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of

removeSpecialChars(df, 'Major', chars=['!','#'])
print(len(df['Major'].unique()))
save_unique_values_to_file(df, 'Major', 'processing/major.txt')

In [ ]:
fuzzyMatches = findFuzzyGroups(df, 'Major', threshold=0.8)
print(f"Similar majors to {fuzzyMatches}")

In [ ]:
df['Major'] = df['Major'].replace({
    'comp. science': 'computer science',
    'enginering': 'engineering',
    'econnomics': 'economics',
}).fillna('Other Majors')

capitalize_words(df, "Major")
save_unique_values_to_file(df, 'Major', 'processing/major.txt')
df['Major'].describe()

The Major column is clean now

### **Gender Column**

In [ ]:
display(df['Gender'].describe())
display(df['Gender'].unique())

***The Gender column has minor typos, capitalziation issues or special characters in strings. Its easy to clean this column .***

In [ ]:
str_basicclean(df,'Gender',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of

removeSpecialChars(df, 'Gender', chars=['!','#'])
df['Gender'] = df['Gender'].fillna('Unknown').astype(str)
capitalize_words(df, "Gender")
save_unique_values_to_file(df, 'Gender', 'processing/gender.txt')

In [ ]:
df['Gender'].describe()

That was quick cleaning steps to clean Gender column. Now the Gender column is clean

### **Scholarship_Status Column**

In [ ]:
display(df['Scholarship_Status'].describe())
display(df['Scholarship_Status'].unique())
save_unique_values_to_file(df, 'Scholarship_Status', 'processing/scholarship_status.txt')

Scholarship_Status column has same problems as Gender column. Sme steps will be applied directly.

In [ ]:
str_basicclean(df,'Scholarship_Status',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of

removeSpecialChars(df, 'Scholarship_Status', chars=['!','#'])
df['Scholarship_Status'] = df['Scholarship_Status'].fillna('Unknown').astype(str)
capitalize_words(df, "Scholarship_Status")
save_unique_values_to_file(df, 'Scholarship_Status', 'processing/scholarship_status.txt')

Scholarship_Status colum ready to go.

### **Socioeconomic_Status Column**

In [ ]:
display(df['Socioeconomic_Status'].describe())
display(df['Socioeconomic_Status'].unique())
save_unique_values_to_file(df, 'Socioeconomic_Status', 'processing/socioeconomic_status.txt')

Socioeconomic_Status column has exact same problems with Gender and Scholarship_Status columns. Exact steps will be applied.

In [ ]:
str_basicclean(df,'Socioeconomic_Status',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of

removeSpecialChars(df, 'Socioeconomic_Status', chars=['!','#'])
capitalize_words(df, "Socioeconomic_Status")
save_unique_values_to_file(df, 'Socioeconomic_Status', 'processing/socioeconomic_status.txt')
df['Socioeconomic_Status'].unique()

Now we need to make Socioeconomic_Status column  - categorical, ordinal-  type, fill the NaN values with median

In [ ]:
#Lets map the socioeconomic status to numerical values for easier median calculation and filling the NaN values
socioeconomic_mapping = {
    'Low': 1,
    'Middle': 2,
    'High': 3,
    'Nan': 0
}
#map the values
df['ses_num'] = df['Socioeconomic_Status'].map(socioeconomic_mapping)
#calculate median excluding 0 (which represents NaN)
median_ses = df['ses_num'].median()
#replace 0 and NaN with median
df['ses_num'] = df['ses_num'].replace(0, median_ses)
df['ses_num'] = df['ses_num'].fillna(median_ses)
#map back to original string values
reverse_map = {1:'Low', 2:'Middle', 3:'High'}
#finalize the 'Socioeconomic_Status' column
df['Socioeconomic_Status'] = df['ses_num'].round().map(reverse_map)
#drop the temporary numerical column
df = df.drop(columns=['ses_num'])

In [ ]:
df['Socioeconomic_Status'].describe()
df['Socioeconomic_Status'].unique()

In [ ]:
# Set 'Socioeconomic_Status' as an ordered categorical variable dtype
df['Socioeconomic_Status'] = pd.Categorical(
    df['Socioeconomic_Status'],
    categories=['Low', 'Middle', 'High'],
    ordered=True
)


Socioeconomic_Status column cleaned

**PartTime_Job Column**



In [ ]:
df["PartTime_Job"].describe()
df["PartTime_Job"].unique()
save_unique_values_to_file(df, 'PartTime_Job', 'processing/parttime_job.txt')

PartTime_Job colum has same problems as Socioeconomic_Status or Scholarship_Status so same steps will be applied.

In [ ]:
str_basicclean(df,'PartTime_Job',style="low") #Uses custom function str_basicclean to strip the characters, and deal with the problem with some of

removeSpecialChars(df, 'PartTime_Job', chars=['!','#'])
capitalize_words(df, "PartTime_Job")
df['PartTime_Job'] = df['PartTime_Job'].fillna('Unknown').replace('Nan','Unknown').astype(str)
save_unique_values_to_file(df, 'PartTime_Job', 'processing/parttime_job.txt')
df['PartTime_Job'].unique()
df['PartTime_Job'].describe()

Now the PartTime_Job column is clean.

### Study_Hours_per_Week Column

In [ ]:
df['Study_Hours_per_Week'].describe()
df['Study_Hours_per_Week'].unique()
save_unique_values_to_file(df, 'Study_Hours_per_Week', 'processing/study_hours_per_week.txt')

The Study_Hours_per_Week column is continous ratio column. So It must be between 0-168 hours. And we need to fill the NaNs with median

In [ ]:
#The Study_Hours_per_Week column is continous ratio column. So It must be between 0-168 hours. And we need to fill the NaNs with median
df['Study_Hours_per_Week'] = pd.to_numeric(df['Study_Hours_per_Week'], errors='coerce')
df['Study_Hours_per_Week'] = df['Study_Hours_per_Week'].clip(0, 168)
num_filler(df, 'Study_Hours_per_Week', mode='median')
df['Study_Hours_per_Week'].describe()


We've cleaned te "Study_Hours_per_Week" column.

### Tutoring_Hours_per_Week Column

In [ ]:
display(df['Tutoring_Hours_per_Week'].unique())
display(df['Tutoring_Hours_per_Week'].describe())
save_unique_values_to_file(df, 'Tutoring_Hours_per_Week', 'processing/tutoring_hours_per_week.txt')

Only filling the NaN/NA values with median will be enough for this column

In [ ]:
num_filler(df, 'Tutoring_Hours_per_Week', mode='median')
df['Tutoring_Hours_per_Week'].describe()

### Attendance_Rate Column

In [ ]:
display(df['Attendance_Rate'].unique())
display(df['Attendance_Rate'].describe())
save_unique_values_to_file(df, 'Attendance_Rate', 'processing/attendance_rate.txt')

Only filling missing values will be enough for "Attendance_Rate" column as well

In [ ]:
num_filler(df, 'Attendance_Rate', mode='median')
df['Attendance_Rate'].describe()

### Sleep_Hours_per_Night Column

In [ ]:
display(df['Sleep_Hours_per_Night'].unique())
display(df['Sleep_Hours_per_Night'].describe())
save_unique_values_to_file(df, 'Sleep_Hours_per_Night', 'processing/sleep_hours_per_night.txt')

In [ ]:
num_filler(df, 'Sleep_Hours_per_Night', mode='median')
df['Sleep_Hours_per_Night'].describe()

### Prior_GPA Column

In [ ]:
display(df['Prior_GPA'].unique())
display(df['Prior_GPA'].describe())
save_unique_values_to_file(df, 'Prior_GPA', 'processing/prior_gpa.txt')

In [ ]:
num_filler(df, 'Prior_GPA', mode='median')
df['Prior_GPA'].describe()

### Exam_Score Column

In [ ]:
display(df['Exam_Score'].unique())
display(df['Exam_Score'].describe())
save_unique_values_to_file(df, 'Exam_Score', 'processing/exam_score.txt')

In [ ]:
num_filler(df, 'Exam_Score', mode='median')
df['Exam_Score'].describe()

## **Export the Clean Data**

In [ ]:
df.to_csv('cleaned_data.csv', index=False)
print("Cleaned data has been exported to 'cleaned_data.csv'.")

# **Visualization**

## **Import the Clean Data**

In [ ]:
df= pd.read_csv('cleaned_data.csv')
df_copy = df.copy()

df.head()

In [ ]:
df.describe().transpose().round(3)